# 便利なコマンド集

インスタンスの時間確認用のコマンド。1.5h触ってないとセッションが切れる。12hで何が何でもリセットされるので注意

In [ ]:
!cat /proc/uptime | awk '{print $1 /60 /60 "hours"}'

CPU/GPUモードの確認。''ならCPUのみ、'/device:GPU:0'ならGPUあり

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

フォルダ展開用のコマンド

In [ ]:
!unzip ./aaa.zip

フォルダZIP用のコマンド

In [ ]:
!zip ./bbb.zip ./bbb/*.png

フォルダ内全削除用のコマンド

In [ ]:
!rm ./ccc/*

# 必要なパッケージのインストール

In [ ]:
!pip install vcopt

# 氷の床パズルを解くクラス

In [8]:
import numpy as np
import numpy.random as nr
from PIL import Image
import matplotlib.pyplot as plt
from copy import deepcopy

class ice_maze():
    def __init__(self, f):
        #フィールドをndarray形式に変換して初期状態とする
        f = np.array([list(f[i]) for i in range(len(f))])
        f[f=='-'] = 0 #氷の床
        f[f=='@'] = 1 #岩・壁
        f[f=='S'] = 2 #サトシ
        f[f=='G'] = 9 #ゴール
        state = np.array(f, int)
        
        #初期状態を記録、shape=(n, tate, yoko)
        self.states = np.array([state], int)
        #状態数の確認
        print('len(self.states) = {}'.format(len(self.states)))
        
        #移動深さの記録
        self.ds = np.array([0], int)
    
    #状態群に同じ状態がないか調べて追加する関数
    def append_state(self, state, d_new):
        #既存の状態と照合した配列、[False, False, True, False, ...]
        ###print(np.max(self.states - state_, axis=(1,2)) == 0)
        #全部Falseなら追加
        if np.sum((np.max(self.states - state, axis=(1,2)) == 0)) == 0: 
            self.states = np.append(self.states, np.array([state], int), axis=0)
            self.ds = np.append(self.ds, np.array([d_new], int), axis=0)
    
    #ゴールした状態を取得する関数
    def get_goal_state(self):
        #各状態の9の数、[1, 1, 1, 1, 0, 1, ...]
        print('num 9 = {}'.format(np.sum(self.states == 9, axis=(1,2))))
        print('depth = {}'.format(self.ds))
        #0個の状態がゴール済み
        goal_index = np.argmax(np.sum(self.states == 9, axis=(1,2)) == 0)
        #ゴールした状態と移動深さを返す
        if goal_index == 0: #ゴールした状態がない場合
            return None, None
        else:
            return self.states[goal_index], self.ds[goal_index]
    
    #ある状態を１ステップ進める関数
    def step_one(self, state, d):
        #ゴールしていたら何もしない
        if np.sum(state == 9) == 0:
            return None
        #現在座標
        x, y = np.where(state==2)[0][0], np.where(state==2)[1][0]
        #上に動ける場合
        if state[x-1, y] != 1:
            #状態をコピー
            state_, x_, y_ = deepcopy(state), deepcopy(x), deepcopy(y)
            #岩等に当たるまで上に移動
            while state_[x_-1, y_] != 1: x_ -= 1
            #移動後の状態
            state_[x, y], state_[x_, y_] = 0, 2
            #状態群に同じ状態がないか調べて追加する
            self.append_state(state_, d + 1)
        #下に動ける場合
        if state[x+1, y] != 1:
            state_, x_, y_ = deepcopy(state), deepcopy(x), deepcopy(y)
            while state_[x_+1, y_] != 1: x_ += 1
            state_[x, y], state_[x_, y_] = 0, 2
            self.append_state(state_, d + 1)
        #左に動ける場合
        if state[x, y-1] != 1:
            state_, x_, y_ = deepcopy(state), deepcopy(x), deepcopy(y)
            while state_[x_, y_-1] != 1: y_ -= 1
            state_[x, y], state_[x_, y_] = 0, 2
            self.append_state(state_, d + 1)
        #右に動ける場合
        if state[x, y+1] != 1:
            state_, x_, y_ = deepcopy(state), deepcopy(x), deepcopy(y)
            while state_[x_, y_+1] != 1: y_ += 1
            state_[x, y], state_[x_, y_] = 0, 2
            self.append_state(state_, d + 1)
    
    #すべての状態を１ステップ進める関数
    def step_all(self):
        for i in range(len(self.states)):
            self.step_one(self.states[i], self.ds[i])
        #状態数の確認
        print('len(self.states) = {}'.format(len(self.states)))
    
    #新しく状態が追加されなくなるまで探索を行う関数
    def step_all_infinite(self):
        num, num_save = -1, -2
        while num != num_save:
            num_save = len(self.states)
            self.step_all()
            num = len(self.states)
    
    def show(self, num=0):
        #素材読み込み
        ice = Image.open('ice.jpg'); ice.close
        rock = Image.open('rock.jpg'); rock.close
        ball = Image.open('ball.jpg'); ball.close
        satoshi = Image.open('satoshi.jpg'); satoshi.close
        #画像作成
        img = Image.new('RGB', (24*len(self.states[0, 0]), 24*len(self.states[0])), (0, 0, 0))
        for i in range(len(self.states[0])):
            for j in range(len(self.states[0, 0])):
                if self.states[num, i, j] == 0:
                    img.paste(ice, (24*j, 24*i))
                if self.states[num, i, j] == 1:
                    img.paste(rock, (24*j, 24*i))
                if self.states[num, i, j] == 9:
                    img.paste(ball, (24*j, 24*i))
                if self.states[num, i, j] == 2:
                    img.paste(satoshi, (24*j, 24*i))
        #表示
        plt.figure(figsize=(8, 8))
        plt.imshow(img)
        plt.show(), plt.close()

# クラスの動作を確認

氷の床パズルの書き方
*  -　→　氷の床（内部番号0）
*  @　→　岩・壁（内部番号1）
*  S　→　サトシ（内部番号2）
*  G　→　ゴール（内部番号9）

In [ ]:
#氷の床パズルを定義
f = ['@@@@@@@@@@@@@@@@',
     '@--------@-----@',
     '@---@----------@',
     '@---------@----@',
     '@-@------------@',
     '@@-------@-----@',
     '@-------------@@',
     '@------@-------@',
     '@--@----------G@',
     '@-------------@@',
     '@-------@------@',
     '@-----@---@----@',
     '@-------------S@',
     '@@@@@@@@@@@@@@@@']

#インスタンスの生成（パズル問題を与えること）
print('インスタンスの生成')
maze = ice_maze(f)

#現在の状態を確認
print('現在の全状態を確認')
print(maze.states.shape)
print(maze.states)

#全状態を1ステップ進める
print('全状態を1ステップ進める')
maze.step_all()

#現在の状態を確認
print('現在の全状態を確認')
print(maze.states.shape)
print(maze.states)

#ゴールした状態を取得
print('ゴールした状態を取得')
goal_state, goal_d = maze.get_goal_state()
print(goal_state, goal_d)

#全状態を終了まで進めてゴールした状態を取得（終了：新規の状態が見つからなくなったら）
print('全状態を終了まで進める')
maze.step_all_infinite()

#ゴールした状態を取得
print('ゴールした状態を取得')
goal_state, goal_d = maze.get_goal_state()
print(goal_state, goal_d)

In [ ]:
#初期状態を描画
print('初期状態を描画')
maze.show()

# 評価関数の作成

フィールドと外壁はあらかじめ用意しておく。

遺伝子長は岩の個数の2倍とし、次のように各岩の(x, y)座標に対応させる。
(画面左上が原点、xは下方向、yは右方向)

例

[5, 3, 7, 1, 2, 2] → [[5, 3], [7, 1], [2, 2]] → 3個の岩を [x, y] = [5, 3], [7, 1], [2, 2] に置く。


xには1～12、yには1～13の整数値が入り得る。座標が重複した岩は1個に重なる。

遺伝子に従って岩を配置した後、パズルを解き、「とり得る全状態の数」×「ゴールに必要な移動深さ」をスコアとする。

スコアが大きくなるように最適化する。



In [11]:
#評価関数
def maze_score(para):
    '''
    関数名は自由だが、paraを受け取るように設計する。
    paraはndarray形式の配列である。
    最後にスコアを返すように設計する。
    '''

    #外枠
    f = ['@@@@@@@@@@@@@@@@',
         '@--------------@',
         '@--------------@',
         '@--------------@',
         '@--------------@',
         '@--------------@',
         '@-------------@@',
         '@--------------@',
         '@-------------G@',
         '@-------------@@',
         '@--------------@',
         '@--------------@',
         '@-------------S@',
         '@@@@@@@@@@@@@@@@']
    
    #迷路のインスタンス作成
    maze = ice_maze(f)
    
    #paraに応じて岩を配置
    para = para.reshape(-1, 2)
    for x, y in para:
        maze.states[0, x, y] = 1 #0番目の状態に岩を配置していく
    
    #初期状態を描画？
    maze.show()

    #迷路を解く
    maze.step_all_infinite()
    
    #ゴールした状態とその移動深さを取得
    goal_state, goal_d = maze.get_goal_state()

    #状態の数を取得
    state_num = len(maze.states)
    
    #ゴールしていなければ（＝ゴールできなければ）スコア0
    if goal_state is None:
        return 0
    
    #そうでなければ「とり得る全状態の数」×「ゴールに必要な移動深さ」をスコアとする
    return state_num * goal_d

In [ ]:
#適当な遺伝子を作成
para = np.array([5, 3, 7, 1, 2, 2], int)
#評価値を確認
print(maze_score(para))

In [ ]:
#ランダムに遺伝子を作成
para = nr.randint(1, 13, 6) #1～12の整数を6個
#評価値を確認
print(maze_score(para))

# 遺伝的アルゴリズムでパズルが難しくなるように遺伝子を最適化

最適化中に出力が大量に出てしまわないように、クラス内のprintすべてと、評価関数内のmaze.show()をコメントアウトする。一つ上のテストコードを実行したときに評価値以外に何も出力されなければOK。

離散的な遺伝子の最適化なので、vcoptの中のdcGA()を使う。

→ [vcoptチュートリアル](https://vigne-cla.com/vcopt-tutorial/)

→ [vcopt仕様書](https://vigne-cla.com/vcopt-specification/)




In [ ]:
from vcopt import vcopt

#パラメータ範囲を指定する配列を用意
para_range = []
for i in range(5): #岩の数
    para_range.append([j for j in range(1, 13)]) #偶数番目の遺伝子はフィールドの縦幅を考慮すると1～12しか取り得ない
    para_range.append([j for j in range(1, 14)]) #奇数番目の遺伝子はフィールドの横幅を考慮すると1～13しか取り得ない
print('para_range:\n{}'.format(para_range))

#GAで最適化
para, score = vcopt().dcGA(para_range,              #パラメータ範囲
                           maze_score,              #評価関数
                           999999,                  #目標値はよくわからないのでクソでかい数。実は'>2000'といった表記も可になった
                           #pool_num=100,           #個体数はデフォルトでは遺伝子長の10倍。指定しても良い
                           max_gen=2000,            #何世代まで進化させるか（※すみません。必ず指定してください。この問題だと永久に終わらない仕様です）
                           show_pool_func='print')  #簡易出力機能を使う


# 最適化した遺伝子でパズル問題を表示

In [ ]:
#最適化後のparaを指定
para = np.array([8, 10, 1, 11, 1, 4, 8, 1, 7, 7])

#外枠
f = ['@@@@@@@@@@@@@@@@',
     '@--------------@',
     '@--------------@',
     '@--------------@',
     '@--------------@',
     '@--------------@',
     '@-------------@@',
     '@--------------@',
     '@-------------G@',
     '@-------------@@',
     '@--------------@',
     '@--------------@',
     '@-------------S@',
     '@@@@@@@@@@@@@@@@']

#迷路のインスタンス作成
maze = ice_maze(f)
    
#paraに応じて岩を配置
para = para.reshape(-1, 2)
for x, y in para:
    maze.states[0, x, y] = 1 #0番目の状態に岩を配置していく

#初期状態を描画？
maze.show()

# 画像を添付してツイートしよう

[クリックでTwitterへジャンプ](https://twitter.com/intent/tweet?text=遺伝的アルゴリズムで氷の床パズルの難問を作ったよ！&hashtags=遺伝的アルゴリズム,vcopt&url=https://aifashion.connpass.com/event/194314/)